In [1]:
import os
import json
import csv
import pandas as pd
import numpy as np
import math

In [2]:
results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_3_2023_merged_full_.csv"

In [3]:
exp_df = pd.read_csv(results_csv_file)  

In [14]:
exp_df

,exp,round0_val,round0_test,num_models,round1_val,round1_test,num_models.1,round2_val,round2_test,num_models.2,round3_val,round3_test,num_models.3,round4_val,round4_test,num_models.4,round5_val,round5_test,num_models.5,round6_test,num_models.6,round7_test,num_models.7,round8_test,num_models.8,round9_test,num_models.9,round10_test,num_models.10,round11_test,num_models.11,round12_test,num_models.12,round13_test,num_models.13,round14_test,num_models.14,round15_test,num_models.15
0,DMPLS_exp2_random_v8,0.673152,0.644054,5,0.747033,0.721900,5,0.768265,0.764608,5,0.778398,0.790210,5,0.755359,0.770185,5,0.790605,0.810495,5,0.797812,0.821120,5.0,0.791092,0.831377,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DMPLS_exp3_random_v8,0.764275,0.793310,5,0.788162,0.797061,5,0.794836,0.805817,5,0.808802,0.829310,5,0.806234,0.815820,5,0.820772,0.830469,5,0.828534,0.837235,5.0,0.815960,0.832590,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DMPLS_exp4_random_v8,0.730566,0.743546,5,0.740578,0.749707,5,0.732175,0.737136,5,0.766462,0.770348,5,0.741282,0.768364,5,0.784812,0.800737,5,0.798883,0.807999,5.0,0.789064,0.805110,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DMPLS_exp0_coreset_v8,0.712811,0.776522,5,0.801925,0.838409,5,0.806550,0.838900,5,0.813218,0.839359,4,0.713384,0.766059,5,0.733775,0.801488,5,0.764997,0.801477,5.0,0.734523,0.785586,5.0,0.782700,0.830308,5.0,0.812316,0.832697,5.0,0.797245,0.836667,5.0,0.808947,0.830860,5.0,NaN,NaN
4,DMPLS_exp1_coreset_v8,0.690875,0.675768,5,0.811622,0.843510,5,0.816157,0.848130,5,0.801801,0.839340,5,0.735381,0.760635,5,0.738242,0.793980,5,0.759405,0.805165,5.0,0.752099,0.806530,5.0,0.768770,0.821878,5.0,0.787378,0.821674,5.0,0.795956,0.832435,5.0,0.804791,0.834785,5.0,NaN,NaN
5,DMPLS_exp2_coreset_v8,0.717027,0.727019,5,0.820484,0.847361,5,0.816445,0.847117,5,0.811664,0.837827,5,0.759269,0.778951,5,0.743477,0.752973,5,0.787476,0.802922,5.0,0.776899,0.804624,5.0,0.799090,0.821146,5.0,0.805126,0.821357,5.0,0.814508,0.834349,5.0,0.811225,0.846940,5.0,NaN,NaN
6,DMPLS_exp3_coreset_v8,0.758788,0.793444,5,0.815706,0.839472,5,0.828916,0.849022,5,0.812771,0.834993,4,0.781475,0.798937,5,0.788928,0.796745,5,0.783619,0.816295,5.0,0.789465,0.801190,5.0,0.810641,0.836719,5.0,0.804958,0.836700,5.0,0.819620,0.835393,5.0,0.821485,0.836172,5.0,NaN,NaN
7,DMPLS_exp0_coreset_fuse10_extra_loss_wt02_use_...,0.743241,0.769006,5,0.735239,0.773450,5,0.761403,0.797265,5,0.787224,0.817631,5,0.757879,0.813974,5,0.798391,0.836225,5,0.796799,0.838974,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,DMPLS_exp1_coreset_fuse10_extra_loss_wt02_use_...,0.728566,0.770216,5,0.770190,0.803232,5,0.736667,0.778669,5,0.770157,0.822782,5,0.763614,0.799952,5,0.786650,0.824119,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,DMPLS_exp2_coreset_fuse10_extra_loss_wt02_use_...,0.725236,0.693117,5,0.779934,0.789758,5,0.775181,0.795487,5,0.789437,0.798411,5,0.787985,0.794262,5,0.805192,0.820893,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
included_exp_names = exp_df['exp'].tolist()

In [5]:
def get_base_exp_name(exp_name):
    exp_field = "exp"
    exp_field_len_w_index = len(exp_field)+1
    exp_field_index = exp_name.find(exp_field)
    exp_name = exp_name[:exp_field_index]+exp_name[exp_field_index+exp_field_len_w_index:]
    return exp_name

In [6]:
exp_groups = dict()

for exp_name in included_exp_names:
    base_exp_name = get_base_exp_name(exp_name)
    if base_exp_name in exp_groups:
        exp_groups[base_exp_name] += [exp_name]
    else:
        exp_groups[base_exp_name] = [exp_name]

In [7]:
exp_length = 5
num_rounds = 16
round_strings =  ["round" + str(round_num) + "_test" for round_num in range(num_rounds)]
base_exp_names = list(exp_groups.keys())

In [8]:
results_dict = {}

for base_exp_name, exp_group in exp_groups.items():
    exp_dict = {}
    for i, run in enumerate(exp_group):
        sub_df = exp_df.loc[exp_df['exp'] == run]
        for round_string in round_strings:
            val = sub_df[round_string].iloc[0]
            if not math.isnan(val):
                if round_string not in exp_dict:
                    exp_dict[round_string] = [float(val)]
                else:
                    exp_dict[round_string] += [float(val)]
    results_dict[base_exp_name] = exp_dict
    print(base_exp_name)

DMPLS__random_v8
DMPLS__coreset_v8
DMPLS__coreset_fuse10_extra_loss_wt02_use_slice_pos_label2_uncertainty1_v8


In [16]:
results_dict

{'DMPLS__random_v8': {'round0_test': [0.6440535193815652,
   0.7933099603816851,
   0.743545551119432,
   0.7588696082677089,
   0.7966095746717355],
  'round1_test': [0.7218999254364609,
   0.7970614227538491,
   0.7497067684099673,
   0.8462995444234139,
   0.8425070759405792],
  'round2_test': [0.7646084597714858,
   0.8058167008425374,
   0.7371356605501317,
   0.8256495870857599,
   0.8381284622345401],
  'round3_test': [0.7902104777459077,
   0.8293096875335495,
   0.7703477119661558,
   0.8404453257297065,
   0.829095493143341],
  'round4_test': [0.7701853218117413,
   0.8158196117353501,
   0.7683644230320178,
   0.794120837916957,
   0.7927785417204952],
  'round5_test': [0.8104948031901289,
   0.830469205078169,
   0.8007367954342629,
   0.7951952388665102,
   0.7911628926292423],
  'round6_test': [0.7978122248673257,
   0.828533971632397,
   0.7988828031541555,
   0.7567243327468737,
   0.8030601227929673],
  'round7_test': [5.0, 5.0, 5.0, 5.0, 5.0],
  'round8_test': [0.8313

In [9]:
index = pd.Index(base_exp_names, name="exp_name")
round_counts_strings = [[round_string, f"round{i}_count"] for i, round_string in enumerate(round_strings)]
df = pd.DataFrame(index=index, columns=sum(round_counts_strings, []))

In [10]:
for base_exp_name, results in results_dict.items():
    sub_df = df.loc[base_exp_name]
    results = results_dict[base_exp_name]
    for round_string, count_string in round_counts_strings:
        if round_string in results:
            round_results = results[round_string]
            num_results = len(round_results)
            mean_result = np.mean(round_results)
            std_results = np.std(round_results, ddof=1)
            mean_string = '%.3f' % mean_result
            std_string = '%.3f'% std_results
            sub_df[round_string] = f"{mean_string}+/-{std_string}"
            sub_df[count_string] = num_results

/Users/arvin/.virtualenvs/al/lib/python3.9/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/arvin/.virtualenvs/al/lib/python3.9/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [12]:
pd.options.display.max_columns = 500 

In [13]:
df

,round0_test,round0_count,round1_test,round1_count,round2_test,round2_count,round3_test,round3_count,round4_test,round4_count,round5_test,round5_count,round6_test,round6_count,round7_test,round7_count,round8_test,round8_count,round9_test,round9_count,round10_test,round10_count,round11_test,round11_count,round12_test,round12_count,round13_test,round13_count,round14_test,round14_count,round15_test,round15_count
exp_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DMPLS__random_v8,0.747+/-0.062,5,0.791+/-0.055,5,0.794+/-0.042,5,0.812+/-0.030,5,0.788+/-0.020,5,0.806+/-0.016,5,0.797+/-0.026,5,5.000+/-0.000,5,0.815+/-0.015,5,0.779+/-0.019,2,5.000+/-0.000,2,0.825+/-0.003,2,0.798+/-0.002,2,5.000+/-0.000,2,0.835+/-0.005,2,NaN,NaN
DMPLS__coreset_v8,0.749+/-0.048,5,0.844+/-0.005,5,0.843+/-0.008,5,0.839+/-0.003,5,0.776+/-0.015,5,0.784+/-0.020,5,0.777+/-0.013,5,5.000+/-0.000,5,0.800+/-0.008,5,0.791+/-0.016,5,5.000+/-0.000,5,0.830+/-0.008,5,0.807+/-0.010,5,5.000+/-0.000,5,0.836+/-0.006,5,NaN,NaN
DMPLS__coreset_fuse10_extra_loss_wt02_use_slice_pos_label2_uncertainty1_v8,0.756+/-0.038,5,0.793+/-0.012,5,0.790+/-0.012,5,0.813+/-0.010,5,0.804+/-0.011,5,0.823+/-0.008,5,0.797+/-nan,1,1.000+/-nan,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
